# 🚀 RoadDoctor: YOLOv8 Training on RDD2022

**Simple & Production-Ready Training Pipeline**

## 📋 Setup (5 minutes):

1. **Kaggle Settings**:
   - Accelerator: **GPU T4 x2** ⚡
   - Internet: **ON** 🌐

2. **Add Dataset**:
   - Click **"+ Add Data"**
   - Search: **"RDD2022"** or use: https://www.kaggle.com/datasets/nirmalsankalana/rdd2022
   - Add dataset (already in YOLO format!)

3. **Run**:
   - Cell → Run All
   - Wait 2-3 hours ☕
   - Download `best.pt`

---

**✨ Features:**
- ✅ Resume training if crashed
- ✅ Auto-save every 5 epochs
- ✅ Early stopping (patience=10)
- ✅ Full metrics & visualizations
- ✅ Handles RDD_SPLIT subdirectory automatically
- ✅ Detects 5 road damage types

## 📦 Step 1: Install & Setup

In [ ]:
# Install Ultralytics YOLOv8
!pip install ultralytics -q

import os
import yaml
import shutil
from pathlib import Path
from datetime import datetime
from ultralytics import YOLO

# Patch imread to skip corrupt images
import cv2
import numpy as np
from ultralytics.utils import patches

original_imread = patches.imread

def safe_imread(filename, flags=cv2.IMREAD_COLOR):
    """Wrapper that returns black image on error instead of crashing"""
    try:
        result = original_imread(filename, flags)
        if result is None:
            # Return small black placeholder
            return np.zeros((100, 100, 3), dtype=np.uint8)
        return result
    except Exception as e:
        # Return small black placeholder on any error
        return np.zeros((100, 100, 3), dtype=np.uint8)

# Apply patch
patches.imread = safe_imread

print("✓ Dependencies installed")
print("✓ Error handling enabled (corrupt images will be skipped)")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Working directory: {os.getcwd()}")

## 🗂️ Step 2: Locate RDD2022 Dataset

In [ ]:
# Find RDD2022 dataset (Kaggle auto-mounts to /kaggle/input)
possible_paths = [
    '/kaggle/input/rdd2022',
    '/kaggle/input/road-damage-dataset-2022',
    '/kaggle/input/rdd-2022',
    '/kaggle/input/rdd2022-dataset'
]

rdd_base_path = None
for path in possible_paths:
    if os.path.exists(path):
        rdd_base_path = Path(path)
        break

if not rdd_base_path:
    raise FileNotFoundError(
        "❌ RDD2022 not found!\n"
        "Please add dataset:\n"
        "1. Click '+ Add Data'\n"
        "2. Search 'RDD2022'\n"
        "3. Add to notebook"
    )

# Check if dataset has RDD_SPLIT subdirectory
if (rdd_base_path / 'RDD_SPLIT').exists():
    rdd_path = rdd_base_path / 'RDD_SPLIT'
    print(f"✓ Found RDD2022 at: {rdd_base_path}")
    print(f"✓ Using split data from: {rdd_path}\n")
else:
    rdd_path = rdd_base_path
    print(f"✓ Found RDD2022 at: {rdd_path}\n")

!ls -lh {rdd_path}

## 📊 Step 3: Verify Dataset Structure

In [ ]:
# Check dataset structure
print("Checking dataset structure...\n")

for split in ['train', 'val', 'test']:
    images_dir = rdd_path / split / 'images'
    labels_dir = rdd_path / split / 'labels'
    
    if images_dir.exists():
        num_images = len(list(images_dir.glob('*.jpg'))) + len(list(images_dir.glob('*.png')))
        num_labels = len(list(labels_dir.glob('*.txt')))
        
        print(f"✓ {split:5s}: {num_images:5d} images, {num_labels:5d} labels")
    else:
        print(f"⚠️  {split:5s}: Not found")

print("\n✓ Dataset verified")

In [ ]:
# RDD2022 classes (dataset has 5 classes, not 4!)
CLASS_NAMES = {
    0: 'longitudinal_crack',
    1: 'transverse_crack',
    2: 'alligator_crack',
    3: 'pothole',
    4: 'other_damage'  # Additional damage types in dataset
}

# Create YAML config
# Use relative paths from rdd_path
dataset_config = {
    'path': str(rdd_path),
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': 5,  # 5 classes, not 4!
    'names': CLASS_NAMES
}

config_path = Path('/kaggle/working/rdd2022.yaml')
with open(config_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print("✓ YAML configuration created\n")
print("Config:")
!cat {config_path}

## 📝 Step 4: Create YOLO Configuration

In [ ]:
# Quick preview - uncomment to see samples
# import matplotlib.pyplot as plt
# import cv2
# import numpy as np

# sample_images = list((rdd_path / 'train' / 'images').glob('*.jpg'))[:6]

# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.flatten()

# for idx, img_path in enumerate(sample_images):
#     img = cv2.imread(str(img_path))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     axes[idx].imshow(img)
#     axes[idx].set_title(img_path.name)
#     axes[idx].axis('off')

# plt.tight_layout()
# plt.show()

print("✓ Ready to train!")

## 👁️ Step 5: Preview Sample Images (Optional)

In [ ]:
# Check for existing checkpoint
checkpoint_path = Path('runs/train/road_defect_detector/weights/last.pt')
resume_training = False

if checkpoint_path.exists():
    print("\n" + "="*60)
    print("⚠️  Found existing checkpoint!")
    print("="*60)
    print(f"Checkpoint: {checkpoint_path}")
    print("\nThis means training was interrupted before.")
    print("\nOptions:")
    print("  1 = Resume from checkpoint")
    print("  2 = Start fresh (delete checkpoint)\n")
    
    # Note: In Kaggle, this will use default (1)
    # You can manually change if needed
    resume_training = True
    print("Auto-resuming from checkpoint...")
else:
    print("Starting fresh training run...")

## 🚀 Step 6: Train YOLOv8n Model

**This will take ~2-3 hours on T4 GPU**

Settings:
- **50 epochs** (adjust if needed)
- **Batch size 16** (decrease to 8 if OOM)
- **Auto-save** every 5 epochs
- **Early stopping** after 10 epochs without improvement

In [ ]:
# Check for existing checkpoint
checkpoint_path = Path('runs/train/road_defect_detector/weights/last.pt')
resume_training = False

if checkpoint_path.exists():
    print("\n" + "="*60)
    print("⚠️  Found existing checkpoint!")
    print("="*60)
    print(f"Checkpoint: {checkpoint_path}")
    print("\nThis means training was interrupted before.")
    print("\nOptions:")
    print("  1 = Resume from checkpoint")
    print("  2 = Start fresh (delete checkpoint)\n")
    
    # Note: In Kaggle, this will use default (1)
    # You can manually change if needed
    resume_training = True
    print("Auto-resuming from checkpoint...")
else:
    print("Starting fresh training run...")

In [ ]:
# Initialize model
if resume_training:
    model = YOLO(str(checkpoint_path))
    print(f"✓ Loaded checkpoint: {checkpoint_path}")
else:
    model = YOLO('yolov8n.pt')
    print("✓ Loaded pretrained YOLOv8n")

print("\n" + "="*60)
print("🚀 STARTING TRAINING")
print("="*60)
print(f"Estimated time: 2-3 hours")
print(f"Monitor progress below...")
print("="*60 + "\n")

In [ ]:
# Train!
results = model.train(
    # Data
    data=str(config_path),
    
    # Training params
    epochs=50,
    imgsz=640,
    batch=16,          # Decrease to 8 if out of memory
    device=0,          # Use GPU 0
    amp=False,         # Disable AMP to avoid Kaggle errors
    workers=4,         # Multi-threaded dataloader (now safe after cleanup)
    
    # Output
    project='runs/train',
    name='road_defect_detector',
    exist_ok=True,
    resume=resume_training,
    
    # Checkpointing
    save=True,
    save_period=5,     # Save every 5 epochs
    patience=10,       # Early stopping
    
    # Data loading
    rect=False,        # Disable rectangular training
    cache=False,       # Don't cache images
    
    # Data augmentation
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
    
    # Optimization
    optimizer='SGD',
    lr0=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    
    # Output
    verbose=True,
    plots=True
)

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)

## 📊 Step 8: Evaluate Model

In [ ]:
# Load best model
best_model_path = 'runs/train/road_defect_detector/weights/best.pt'
model = YOLO(best_model_path)

# Validate
metrics = model.val()

print("\n" + "="*60)
print("📊 MODEL PERFORMANCE METRICS")
print("="*60)
print(f"mAP50:     {metrics.box.map50:.3f}    (main metric)")
print(f"mAP50-95:  {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall:    {metrics.box.mr:.3f}")
print("="*60)

# Expected results:
# mAP50: 0.70-0.80
# Precision: 0.70-0.78
# Recall: 0.65-0.75

## 📊 Step 7: Evaluate Model

In [ ]:
from IPython.display import Image as IPImage, display

results_dir = Path('runs/train/road_defect_detector')

print("\n📊 Training Curves:\n")
if (results_dir / 'results.png').exists():
    display(IPImage(filename=str(results_dir / 'results.png')))

print("\n🎯 Confusion Matrix:\n")
if (results_dir / 'confusion_matrix.png').exists():
    display(IPImage(filename=str(results_dir / 'confusion_matrix.png')))

print("\n📉 F1 Score Curve:\n")
if (results_dir / 'F1_curve.png').exists():
    display(IPImage(filename=str(results_dir / 'F1_curve.png')))

## 📈 Step 8: View Training Results

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Get random test images
test_images = list((rdd_path / 'val' / 'images').glob('*.jpg'))[:8]

print(f"Testing on {len(test_images)} validation images...\n")

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, img_path in enumerate(test_images):
    # Run inference
    results = model(str(img_path), verbose=False)
    
    # Get annotated image
    annotated = results[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    # Plot
    axes[idx].imshow(annotated)
    axes[idx].axis('off')
    
    # Title with detection count
    num_detections = len(results[0].boxes)
    axes[idx].set_title(f"{img_path.name}\n{num_detections} defects", fontsize=10)

plt.tight_layout()
plt.savefig('/kaggle/working/test_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Test predictions shown")
print("✓ Saved to: /kaggle/working/test_predictions.png")

## 🧪 Step 9: Test Predictions

In [ ]:
# Copy model to working directory for easy download
output_model = Path('/kaggle/working/best.pt')
backup_model = Path('/kaggle/working/best_backup.pt')

shutil.copy(best_model_path, output_model)
shutil.copy(best_model_path, backup_model)

# Copy training plots
if (results_dir / 'results.png').exists():
    shutil.copy(results_dir / 'results.png', '/kaggle/working/training_results.png')
if (results_dir / 'confusion_matrix.png').exists():
    shutil.copy(results_dir / 'confusion_matrix.png', '/kaggle/working/confusion_matrix.png')

print("\n" + "="*70)
print("💾 MODEL SAVED!")
print("="*70)
print(f"\nModel files:")
print(f"  • best.pt             ({output_model.stat().st_size / 1024 / 1024:.1f} MB)")
print(f"  • best_backup.pt      ({backup_model.stat().st_size / 1024 / 1024:.1f} MB)")
print(f"\nVisualization files:")
print(f"  • training_results.png")
print(f"  • confusion_matrix.png")
print(f"  • test_predictions.png")
print("\n" + "="*70)
print("📥 HOW TO DOWNLOAD:")
print("="*70)
print("1. Click 'Output' tab on the right panel")
print("2. Download ALL files above")
print("3. Place best.pt in: ml/models/best.pt")
print("="*70)

## 💾 Step 10: Save & Download Model